# Rental Price Calculator Model

## Data

We load the data that we have cleaned up. This is already separated into train and test sets.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
train_df = pd.read_csv("data/clean_rent_data_post_desc_train.csv")
test_df = pd.read_csv("data/clean_rent_data_post_desc_test.csv")

## Scaling The Data

First we will scale the data so that it can be better trained against

We left description in the cleaned data, but we should remove it before scaling because it is not a numeric

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [4]:
train_df = train_df.drop(columns=["description"])
test_df = test_df.drop(columns=["description"])

Let's also do some final data cleaning by dropping some outliers. Through inspection of the data, we see that some rentals are priced over 35000 per month and under 1000 per month. There are very few of these so they probably don't fit the standard pricing model, so we can drop them from the data set.

In [5]:
train_df = train_df[(train_df["price"] > 1000) & (train_df["price"] < 35000)]
test_df = test_df[(test_df["price"] > 1000) & (test_df["price"] < 35000)]

In [6]:
X_train = train_df.drop(columns=["price"])
Y_train = train_df["price"]
X_test = test_df.drop(columns=["price"])
Y_test = test_df["price"]

In [7]:
X_train.shape

(44319, 1228)

### Smaller Training Data
Because we have so many columns, training models can take a lot of time.

We'll further split the training set in order to speed that process up. In order to have fast training, we will split it up 50/50

We will also be looking for the optimal hyper paramters which will further require many calculations which would be benefited by having a smaller training set.

In [8]:
X_train_sm, X_valid, Y_train_sm, Y_valid = train_test_split(X_train, Y_train, test_size=0.5)

In [9]:
scaler = StandardScaler()

# fitting the scaler to the training data and then also transforming the validation and test data
X_train_scaled = scaler.fit_transform(X_train_sm)

# transform the test data without fitting (only fit on the train data)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

In [10]:
# We see that the numbers are now all around -2 to 2 as expected from scaling
X_train_scaled[0]

array([ 1.56175846,  1.29704212, -0.66026   , ..., -0.41074614,
       -0.11741095, -0.07127451])

## Linear Regression

We will try a linear regression first.

In [11]:
from sklearn.linear_model import LinearRegression

lr_model = LinearRegression()
 
# Fit the model to the training data only
lr_model.fit(X_train_scaled, Y_train_sm)

lr_y_pred = lr_model.predict(X_valid_scaled)
 
# scoring the train and test set
print(f'Train score: {lr_model.score(X_train_scaled, Y_train_sm)}')
print(f'Validation score: {lr_model.score(X_valid_scaled, Y_valid)}')



Train score: 0.7336617483233161
Validation score: -3.39783241129853e+20


In [12]:
from sklearn.metrics import mean_squared_error as mse
# calculating the MSE
print(f'MSE: {mse(Y_valid, lr_y_pred)}')
 
# calculating the RMSE
print(f'RMSE: {np.sqrt(mse(Y_valid, lr_y_pred))}')
 
# calculating the MAE
print(f'MAE: {np.mean(np.abs(Y_valid - lr_y_pred))}')

MSE: 1.4455052281119584e+27
RMSE: 38019800474383.85
MAE: 1368997117509.44


The low test score indicates that the model is overfitted on the training data. Let's look at the coefficients to see if they make sense

In [13]:
def print_coefficients_info(coefficients):
    coeff_disp = [[round(coefficients[i]), X_train.columns[i]] for i in range(coefficients.shape[0])]
    print(coefficients.shape)
    print(sorted(coeff_disp, key=lambda x: abs(x[0]), reverse=True))

In [14]:
print_coefficients_info(lr_model.coef_)

(1228,)
[[290597770110542, 'zipcode_10028'], [277220247137929, 'zipcode_10606'], [248345695923464, 'zipcode_10017'], [241375143810278, 'zipcode_10018'], [230659918237416, 'zipcode_10009'], [219190271775737, 'zipcode_10011'], [214051097963373, 'zipcode_10044'], [212180294997277, 'zipcode_10003'], [198780128823087, 'zipcode_10016'], [196027553904086, 'zipcode_11413'], [190898872997357, 'zipcode_10019'], [187612291889028, 'zipcode_10199'], [183279325969047, 'zipcode_08515'], [182035844951773, 'zipcode_10021'], [179326575620740, 'zipcode_10012'], [179174722103935, 'zipcode_10023'], [176416497480598, 'zipcode_10162'], [169787268455130, 'zipcode_10002'], [169140953740357, 'zipcode_10025'], [168003326439953, 'zipcode_10271'], [167185531201793, 'zipcode_10001'], [160478693264923, 'zipcode_10065'], [-157291936148962, 'zipcode_11412'], [155257929422137, 'zipcode_08088'], [147193780873749, 'zipcode_10022'], [146570289078940, 'desc_word_ellimancoma'], [143568668418574, 'zipcode_10005'], [143506450

We can see that many of the columns have very large coefficients. Combined with the high training score and the low test score, this likely means there is overfitting, probably on the description words and the zip codes. Let's try to tune which columns we keep from those sets of columns.

Let's drop some of the created columns since they aren't needed besides created_day

In [15]:
more_cols_to_drop = ["created_epoch_secs", "created_year"]
X_train = X_train.drop(columns=more_cols_to_drop)
X_test = X_test.drop(columns=more_cols_to_drop)

Let's remove any description words that show up less often. Through inspection of some of the high impact words and descriptions, we can see that some of the words are due to the syntax of the descriptions rather than something to do with the pricing of the apartment, so it is likely we are overfitting.

In [16]:
desc_counts = X_train.filter(like="desc_word_").sum()

It looks like some of the zip code coefficients are very high. We probably do not have enough data on zip codes to train a model with them, but they might have high impact on the price. Let's try dropping all zip codes under a certain threshold as well.

In [17]:
zipcode_counts = X_train.filter(like="zipcode_").sum()

In [18]:
# possible pairings of desc_count thresholds and zipcode_count_thresholds to try
# the first element is the number of listings that have the description
# the second element is the number of listings that have the zipcode

# Note that I have attempted a lot more combinations, but you may only be able
# to run a few combinations in one session due to memory issues

# desc_count_thresholds = [0, 500, 1000, 1500, 2000, 3000, 5000]
# zipcode_count_thresholds = [0, 10, 15, 20, 50, 80, 100, 200]

desc_count_thresholds = [500, 1000, 1500, 2000]
zipcode_count_thresholds = [10, 15, 20]

new_train_datas = {}

for desc_count_threshold in desc_count_thresholds:
    for zipcode_count_threshold in zipcode_count_thresholds:
        new_train = X_train
        low_count_words = desc_counts[desc_counts < desc_count_threshold].index
        low_count_words.array
        new_train = new_train.drop(columns=low_count_words)

        low_pop_zipcodes = zipcode_counts[zipcode_counts < zipcode_count_threshold].index
        low_pop_zipcodes.array

        new_train = new_train.drop(columns=low_pop_zipcodes)
        new_train_datas["%s_%s" % (desc_count_threshold, zipcode_count_threshold)] = new_train
        
len(new_train_datas)

12

Now let's try the linear regression again with each of the new_train_data sets. We have to scale again first each time

In [19]:
for desc_count_threshold in desc_count_thresholds:
    for zipcode_count_threshold in zipcode_count_thresholds:
        new_train = new_train_datas["%s_%s" % (desc_count_threshold, zipcode_count_threshold)]
        new_train_sm, new_valid, Y_new_train_sm, Y_new_valid = train_test_split(new_train, Y_train, test_size=0.5)
        
        scaler = StandardScaler()

        # fitting the scaler to the training data and then also transforming the validation and test data
        new_train_scaled = scaler.fit_transform(new_train_sm)

        # transform the test data without fitting (only fit on the train data)
        new_valid_scaled = scaler.transform(new_valid)
        
        new_lr = LinearRegression()
 
        # Fit the model to the training data only
        new_lr.fit(new_train_scaled, Y_new_train_sm)

        # scoring the train and test set
        print(f'for: {desc_count_threshold} {zipcode_count_threshold}')
        print(f'Train score: {new_lr.score(new_train_scaled, Y_new_train_sm)}')
        print(f'Validation score: {new_lr.score(new_valid_scaled, Y_new_valid)}')

for: 500 10
Train score: 0.7073569367332089
Validation score: -8.237930382726495e+19
for: 500 15
Train score: 0.7055387038308188
Validation score: -9.766340340223418e+18
for: 500 20
Train score: 0.7267503387769649
Validation score: -8.437711573055596e+19
for: 1000 10
Train score: 0.6928735094991152
Validation score: 0.6833618147570395
for: 1000 15
Train score: 0.6994677288576368
Validation score: 0.6764556000198035
for: 1000 20
Train score: 0.6977593377691005
Validation score: 0.6783224567364514
for: 1500 10
Train score: 0.6903237148839743
Validation score: -1.1359111374837614e+20
for: 1500 15
Train score: 0.6830237284758895
Validation score: -3.5732884301223405e+17
for: 1500 20
Train score: 0.6766823043586001
Validation score: 0.6747534268262652
for: 2000 10
Train score: 0.676127946922569
Validation score: 0.6726571168777167
for: 2000 15
Train score: 0.6927222397886315
Validation score: 0.6573738671622624
for: 2000 20
Train score: 0.6791948034617337
Validation score: 0.669273614601071

We can see that the model performs relatively closely when we remove description words that occur less than 2000 times and zip codes that occur less than 15 times. Through some other trial and error, we choose these to be the thresholds - although on some runs depending on the training set different hyperparameters may be better, it seems like below 2000 sometimes there is a large drop in score. Let's remove the columns from all the original sets of data now.

We will create a new validation set and scale again.

In [20]:
low_count_words = desc_counts[desc_counts < 2000].index
low_count_words.array
low_pop_zipcodes = zipcode_counts[zipcode_counts < 15].index
low_pop_zipcodes.array

X_train = X_train.drop(columns=low_count_words)
X_train = X_train.drop(columns=low_pop_zipcodes)
X_test = X_test.drop(columns=low_count_words)
X_test = X_test.drop(columns=low_pop_zipcodes)

In [21]:
X_train_sm, X_valid, Y_train_sm, Y_valid = train_test_split(X_train, Y_train, test_size=0.5)

scaler = StandardScaler()

# fitting the scaler to the training data and then also transforming the validation and test data
X_train_scaled = scaler.fit_transform(X_train_sm)

# transform the test data without fitting (only fit on the train data)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

After tuning which columns we remove, we have a reasonable model, but it is still not that good. Let's see if we can improve it by tuning other hyper parameters. Linear regressions don't have that many parameters so this might not help that much.

The hyper parameter tuning will do a 5 fold cross validation.

In [22]:
from sklearn.model_selection import GridSearchCV

params = {
    'fit_intercept': [True, False],
    'normalize': [True, False],
    'copy_X': [True, False]
}
 
grid = GridSearchCV(lr_model, params, cv=5)
 
grid.fit(X_train_scaled, Y_train_sm)
 
grid.best_params_

{'copy_X': True, 'fit_intercept': True, 'n_jobs': -1, 'normalize': False}


In [23]:
# instantiate a new model with the best parameters
lr_model = LinearRegression(copy_X=True, fit_intercept=True, n_jobs=-1, normalize=False)
 
# Fit the model to the training data only
lr_model.fit(X_train_scaled, Y_train_sm)

lr_y_pred = lr_model.predict(X_valid_scaled)
 
# scoring the train and test set
print(f'Train score: {lr_model.score(X_train_scaled, Y_train_sm)}')
print(f'Validation score: {lr_model.score(X_valid_scaled, Y_valid)}')
 

Train score: 0.67380441388275
Validation score: 0.6748891355428662


In [24]:

# calculating the MSE
print(f'MSE: {mse(Y_valid, lr_y_pred)}')

# calculating the RMSE
print(f'RMSE: {np.sqrt(mse(Y_valid, lr_y_pred))}')
 
# calculating the MAE
print(f'MAE: {np.mean(np.abs(Y_valid - lr_y_pred))}')

MSE: 1427154.331623955
RMSE: 1194.63564806344
MAE: 700.8383135897212


Let's inspect the model to see how different factors might impact rental price

In [46]:
print_coefficients_info(lr_model.coef_)

(364,)
[[977, 'bathrooms'], [643, 'bedrooms'], [402, 'zipcode_10011'], [341, 'zipcode_10012'], [320, 'zipcode_10003'], [292, 'zipcode_10009'], [271, 'zipcode_10028'], [265, 'zipcode_10017'], [263, 'zipcode_10023'], [262, 'zipcode_10001'], [251, 'zipcode_10019'], [244, 'zipcode_10021'], [243, 'zipcode_10014'], [228, 'zipcode_10065'], [225, 'zipcode_10018'], [223, 'doorman'], [220, 'zipcode_10016'], [214, 'zipcode_10002'], [208, 'zipcode_10044'], [205, 'zipcode_10162'], [201, 'zipcode_10199'], [189, 'zipcode_10022'], [-177, 'desc_word_support'], [170, 'zipcode_10013'], [159, 'zipcode_10007'], [159, 'zipcode_10271'], [157, 'zipcode_10069'], [137, 'zipcode_10025'], [134, 'zipcode_10005'], [132, 'zipcode_10006'], [-132, 'desc_word_flex'], [121, 'zipcode_10282'], [119, 'desc_word_estat'], [118, 'zipcode_10154'], [116, 'zipcode_10103'], [114, 'laundry_in_unit'], [114, 'zipcode_10278'], [108, 'zipcode_11201'], [107, 'zipcode_10038'], [104, 'zipcode_10171'], [103, 'zipcode_10280'], [98, 'zipcod

### Logistic Regression or KNN

This is not a classification problem so a logistic regression or KNN likely won't be very good.

## Random Forest

The random forest may be a suitable model for rental prices because decision trees are similar to how customers may make decisions and hence set the market.

Here, we do not need to use the scaled data, so we can use X_train directly. We will use X_train_sm for now to avoid using the test data.

In [25]:
from sklearn.ensemble import RandomForestRegressor
 
rf_model = RandomForestRegressor(n_jobs=-1, random_state=1)
 
# Fit the model to the training data only
rf_model.fit(X_train_sm, Y_train_sm)

rf_y_pred = rf_model.predict(X_valid)
 


In [26]:

# scoring the train and test set
print(f'Train score: {rf_model.score(X_train_sm, Y_train_sm)}')
print(f'Validation score: {rf_model.score(X_valid, Y_valid)}')

Train score: 0.9744166860101235
Validation score: 0.8266172930801319


In [27]:

# calculating the MSE
print(f'MSE: {mse(Y_valid, rf_y_pred)}')

# calculating the RMSE
print(f'RMSE: {np.sqrt(mse(Y_valid, rf_y_pred))}')
 
# calculating the MAE
print(f'MAE: {np.mean(np.abs(Y_valid - rf_y_pred))}')

MSE: 761106.1587331304
RMSE: 872.4139835726675
MAE: 426.63538385783556


We can see the Random Forest Regressor out performs the linear regression. It has some overfitting. We can tune the hyper parameters even further to try to improve it. Here, we will try a 3-fold and try a grid of different hyper parameters. We choose specifically low number of options and 3-fold instead of the standard 5-fold here to improve run time, but you may try different options to land on the optimal ones.

We can run this on the original X_train because it will generate the validation sets out of that set, hence we do not overfit before looking at the performance on the test set which has not been used to fit anything yet. We don't need to use X_train_sm since it will create the validation set for us.

In [28]:
# 200, 300 and 20, 30 => 300, 30
# 300, 500 and 30, 50 => 500, 50
# creating a dictionary of hyperparameters
params = {
    'n_estimators': [300, 500],
    'max_depth': [30, 50]
}
    
# instantiating the grid search
grid = GridSearchCV(rf_model, params, cv=3)
 
# fitting the grid search
grid.fit(X_train, Y_train)
 
# the best parameters
grid.best_params_

previous runs showed 500, 50 was a good set of parameters


For random forest, typically the higher the number of estimators and max depth, the better, although it may lead to overfitting. However, the higher these values, the longer the model may take to run. Because of limited compute power, we weren't able to test above 500 estimators and 50 depth

In [28]:
from sklearn.ensemble import RandomForestRegressor
 
rf_model = RandomForestRegressor(
    n_estimators=500,
    max_depth=50,
    n_jobs=-1,
    random_state=1
)
 
# Fit the model to the training data only
rf_model.fit(X_train_sm, Y_train_sm)

rf_y_pred = rf_model.predict(X_valid)

# scoring the train and test set
print(f'Train score: {rf_model.score(X_train_sm, Y_train_sm)}')
print(f'Validation score: {rf_model.score(X_valid, Y_valid)}')

Train score: 0.9748450328963889
Validation score: 0.8299247735167214


In [29]:

# calculating the MSE
print(f'MSE: {mse(Y_valid, rf_y_pred)}')

# calculating the RMSE
print(f'RMSE: {np.sqrt(mse(Y_valid, rf_y_pred))}')
 
# calculating the MAE
print(f'MAE: {np.mean(np.abs(Y_valid - rf_y_pred))}')

MSE: 746587.1575311183
RMSE: 864.0527515905023
MAE: 423.5105148359923


## XGBoost

XGBoost is the extreme gradient boosting model. It uses decision trees similar to random forest, but then the models under each decision tree are further boosted by combining models together to form a stronger model.

In [32]:
from xgboost import XGBRegressor
 
xgb = XGBRegressor()

# fitting the model to the training set
xgb.fit(X_train_sm, Y_train_sm)
 
# predicting on the validation set
xgb_y_pred = xgb.predict(X_valid)

print(f'Train score: {xgb.score(X_train_sm, Y_train_sm)}')
print(f'Validation score: {xgb.score(X_valid, Y_valid)}')

Train score: 0.9458181969078618
Validation score: 0.8377872328719473


In [33]:


# calculating the MSE
print(f'MSE: {mse(Y_valid, xgb_y_pred)}')

# calculating the RMSE
print(f'RMSE: {np.sqrt(mse(Y_valid, xgb_y_pred))}')
 
# calculating the MAE
print(f'MAE: {np.mean(np.abs(Y_valid - xgb_y_pred))}')

MSE: 712072.9528312399
RMSE: 843.8441519802338
MAE: 458.932559217873


Let's try some different hyper parameters

In [39]:
params = {
    'n_estimators': [200, 500],
    'learning_rate': [0.25],
    'max_depth': [2, 8]
}
    
grid = GridSearchCV(xgb, params, cv=3)
 
grid.fit(X_train, Y_train)
 
grid.best_params_

{'learning_rate': 0.25, 'max_depth': 8, 'n_estimators': 500}

In [34]:
xgb = XGBRegressor(
    n_estimators=500, 
    learning_rate=0.25,
    max_depth=8
)

# fitting the model to the training data
xgb.fit(X_train_sm, Y_train_sm)
 
# predicting on the test set
xgb_y_pred = xgb.predict(X_valid)

# scoring the train and test set
print(f'Train score: {xgb.score(X_train_sm, Y_train_sm)}')
print(f'Validation score: {xgb.score(X_valid, Y_valid)}')

Train score: 0.9978851963174936
Validation score: 0.8519405820805195


In [40]:


# calculating the MSE
print(f'MSE: {mse(Y_valid, xgb_y_pred)}')

# calculating the RMSE
print(f'RMSE: {np.sqrt(mse(Y_valid, xgb_y_pred))}')
 
# calculating the MAE
print(f'MAE: {np.mean(np.abs(Y_valid - xgb_y_pred))}')

MSE: 649943.3354044943
RMSE: 806.1906321736158
MAE: 404.2215803125705


We might be able to tune the hyper parameters some more, but we see that the train score is close to 1 while the validation score is not at that level. This indicates overfitting which may not be improved greatly by further hyper parameter tuning.

## Best Model

Our best model is the XGBoost model. This makes sense since price may involve decision trees, and the XGBoost model improves on random forest by combining weak models.

Let's see how it performs against the test set

In [45]:
print(f'Test score: {xgb.score(X_test, Y_test)}')

xgb_test_y_pred = xgb.predict(X_test)

print(f'MSE: {mse(Y_test, xgb_test_y_pred)}')
print(f'RMSE: {np.sqrt(mse(Y_test, xgb_test_y_pred))}')
print(f'MAE: {np.mean(np.abs(Y_test - xgb_test_y_pred))}')

Test score: 0.8044522584560098
MSE: 892357.8965946656
RMSE: 944.6469692931141
MAE: 417.72910687594464


As expected, it does a bit worse than the validation set.
Let's save the model so others can use it

In [41]:
import pickle

In [42]:
pickle.dump(xgb, open('nyc_rental_price.pkl','wb')) 

Here we can load the saved model and predict a price with it

In [43]:
saved_model = pickle.load(open('nyc_rental_price.pkl','rb'))

In [44]:
# predicting the price of a vehicle on input data
print(f"Predicted price: {saved_model.predict(X_test)[0]:.2f}")
print(f"Actual price: {Y_test.iloc[0]}")

Predicted price: 4259.35
Actual price: 4300


## Improvements

We can see by the MAE that the model is on average $400 off from the true rental price. Although we may try tuning the model and hyper parameters more, We are likely missing some key pieces of data for NYC rental properties to improve this much further - notably square footage. Likely followups would be to be smarter on grabbing this information from the description, or to find a way to add on more valuable data and then retune the data. A neural network properly tuned may also present some improvements.